# Review DPS Outputs

Make a mosaic of DPS outputs.

1. make a list of the DPS output paths
2. merge that list with the original index tile file
3. convert the local paths to s3 paths
4. reproject the bounds to 4326 - convert to geojson (save file or in mem object)
5. make mosaicjson Mosaic.from_features
6. send mosaicjson to the tile server

In [93]:
import geopandas as gpd
import pandas as pd
import os
import json

## Build a Geojson with all the data

In [19]:
def local_to_s3(url):
    ''' A Function to convert local paths to s3 urls'''
    user = 'nathanmthomas'
    return url.replace('/projects/my-private-bucket', f's3://maap-ops-workspace/{user}')

In [22]:
# We just need a list of files
# TODO: skip making a shapefile from the outputs
outputs = gpd.read_file("/projects/DPS/Landsat_tindex.shp")
output_local = outputs['location'].tolist()
output_local[0:10]

['/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/07/30/00/25/20/674788/Landsat8_986_comp_cog_2015-2020_dps.tif',
 '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/07/30/00/28/08/865666/Landsat8_987_comp_cog_2015-2020_dps.tif',
 '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/07/30/00/29/07/820425/Landsat8_979_comp_cog_2015-2020_dps.tif',
 '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/07/30/00/29/19/175707/Landsat8_984_comp_cog_2015-2020_dps.tif',
 '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/07/30/00/29/25/875834/Landsat8_982_comp_cog_2015-2020_dps.tif',
 '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/07/30/00/29/35/110413/Landsat8_985_comp_cog_2015-2020_dps.tif',
 '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/07/30/00/31/35/031032/Landsat8_980_comp_cog_2015-2020_dp

In [65]:
# Build up a dataframe from the list of dps output files
base_frame = pd.DataFrame(output_local, columns=['local_path'])
base_frame['id'] = [int(os.path.basename(local_path).split('_')[1]) for local_path in base_frame['local_path']]
base_frame['s3'] = [local_to_s3(local_path) for local_path in base_frame['local_path']]

In [66]:
base_frame.sort_values('id', inplace=True)
#base_frame.set_index('id', drop=True, inplace=True)
base_frame.head()


,local_path,id,s3
52,/projects/my-private-bucket/dps_output/do_land...,358,s3://maap-ops-workspace/nathanmthomas/dps_outp...
56,/projects/my-private-bucket/dps_output/do_land...,389,s3://maap-ops-workspace/nathanmthomas/dps_outp...
55,/projects/my-private-bucket/dps_output/do_land...,420,s3://maap-ops-workspace/nathanmthomas/dps_outp...
59,/projects/my-private-bucket/dps_output/do_land...,421,s3://maap-ops-workspace/nathanmthomas/dps_outp...
48,/projects/my-private-bucket/dps_output/do_land...,451,s3://maap-ops-workspace/nathanmthomas/dps_outp...


In [83]:
vector_path = '/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg'
tile_index = gpd.read_file(vector_path)
tile_index['layer'] = tile_index.index
tile_index.rename(columns={"layer":"id"}, inplace=True)
tile_index.head()

,id,geometry
0,0,"POLYGON ((-2151478.000 9423304.000, -2061478.0..."
1,1,"POLYGON ((-2061478.000 9423304.000, -1971478.0..."
2,2,"POLYGON ((-1971478.000 9423304.000, -1881478.0..."
3,3,"POLYGON ((-2241478.000 9333304.000, -2151478.0..."
4,4,"POLYGON ((-2151478.000 9333304.000, -2061478.0..."


In [84]:
# Select the rows we have results for
tile_matches = tile_index.iloc[base_frame['id'].tolist()].merge(base_frame, on='id')

In [85]:
tile_matches.head()

,id,geometry,local_path,s3
0,358,"POLYGON ((4598522.000 7623304.000, 4688522.000...",/projects/my-private-bucket/dps_output/do_land...,s3://maap-ops-workspace/nathanmthomas/dps_outp...
1,389,"POLYGON ((4598522.000 7533304.000, 4688522.000...",/projects/my-private-bucket/dps_output/do_land...,s3://maap-ops-workspace/nathanmthomas/dps_outp...
2,420,"POLYGON ((4508522.000 7443304.000, 4598522.000...",/projects/my-private-bucket/dps_output/do_land...,s3://maap-ops-workspace/nathanmthomas/dps_outp...
3,421,"POLYGON ((4598522.000 7443304.000, 4688522.000...",/projects/my-private-bucket/dps_output/do_land...,s3://maap-ops-workspace/nathanmthomas/dps_outp...
4,451,"POLYGON ((4238522.000 7353304.000, 4328522.000...",/projects/my-private-bucket/dps_output/do_land...,s3://maap-ops-workspace/nathanmthomas/dps_outp...


In [95]:
geojson_string = tile_matches.to_crs("EPSG:4326").to_json()
geojson = json.loads(geojson_string)

In [96]:
geojson.get('features')[10]

{'id': '10',
 'type': 'Feature',
 'properties': {'id': 491,
  'local_path': '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/08/16/20/53/18/088339/Landsat8_491_comp_cog_2015-2020_dps.tif',
  's3': 's3://maap-ops-workspace/nathanmthomas/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/08/16/20/53/18/088339/Landsat8_491_comp_cog_2015-2020_dps.tif'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-53.749997757392386, 53.29128049169117],
    [-54.15496178265806, 52.52477346033019],
    [-55.435091948407276, 52.75709388583292],
    [-55.052643172537486, 53.5277365763123],
    [-53.749997757392386, 53.29128049169117]]]}}

## Build a MosaicJSON

In [89]:
from typing import Dict

from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

In [97]:
def get_accessor(feature: Dict):
    """Return specific feature identifier."""
    return feature["properties"]["s3"]

In [98]:
mosaicdata = MosaicJSON.from_features(geojson.get('features'), minzoom=8, maxzoom=16, accessor=get_accessor)

In [100]:
mosaic_json = "/projects/my-public-bucket/Landsat/landsat8_mosaic.json"
with MosaicBackend(mosaic_json, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)
    
# URL to give tiler is
# s3://maap-ops-workspace/shared/nathanmthomas/Landsat/landsat8_mosaic.json

## View the Results

In [103]:
import ipycmc
import urllib

In [107]:
wmts_url = "https://jqsd6bqdsf.execute-api.us-west-2.amazonaws.com/mosaicjson/WMTSCapabilities.xml"
params = {
    "tile_format":"png",
    "tile_scale":"1",
    "pixel_selection":"first",
    "TileMatrixSetId":"WebMercatorQuad",
    "url":"s3://maap-ops-workspace/shared/nathanmthomas/Landsat/landsat8_mosaic.json",
    "bidx":"6", # Select which band to use
    "resampling_method":"nearest",
    "rescale":"0,1", # Values in data are from 0 to 1
    "return_mask":"true",
    "colormap_name":"magma" # Any colormap from matplotlib will work
}

wmts_call = "?".join([wmts_url, urllib.parse.urlencode(params)])

# Note Jupyter bug add amp; incorrectly when printing the url
wmts_call

'https://jqsd6bqdsf.execute-api.us-west-2.amazonaws.com/mosaicjson/WMTSCapabilities.xml?tile_format=png&tile_scale=1&pixel_selection=first&TileMatrixSetId=WebMercatorQuad&url=s3%3A%2F%2Fmaap-ops-workspace%2Fshared%2Fnathanmthomas%2FLandsat%2Flandsat8_mosaic.json&bidx=6&resampling_method=nearest&rescale=0%2C1&return_mask=true&colormap_name=magma'

In [108]:
w = ipycmc.MapCMC()
w

MapCMC()

In [109]:
# This adds a new layer to the map above, call Cloud Optimized GeoTIFF
w.load_layer_config(wmts_call, "wmts/xml")